# Network Analysis of Functional Connectivity

In [ ]:
pip install networkx

Like last time, make sure you restart the notebook kernel (`Kernel` > `Restart Kernel...`). Once that is done, you can run the next cell to import all of the dependencies for today. 

In [ ]:
import numpy as np
import pandas as pd
from scipy import spatial, stats, cluster
import matplotlib.pyplot as plt
import seaborn as sns
from nilearn import plotting
from nilearn.datasets import fetch_atlas_schaefer_2018
import networkx as nx
import bct

In [ ]:
atlas = fetch_atlas_schaefer_2018(n_rois=100, resolution_mm=2)
labels = [x.decode() for x in atlas['labels']]

## 1. Thresholding

In [ ]:
cmat = np.loadtxt('connectivity.csv', delimiter=',')

fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(cmat, vmax=1, vmin=-1, figure=fig)

In [ ]:
thresh_cmat = bct.threshold_absolute(cmat, thr=.3)

fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(thresh_cmat, vmax=1, vmin=-1, figure=fig)

In [ ]:
bin_cmat = bct.binarize(thresh_cmat)
fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(bin_cmat, vmax=1, vmin=0, cmap='binary', figure=fig)

## 2. Very basics of graph analysis

Let's just take the same subnetwork we looked at last week, and also add in the motor regions. This will make life a bit easier for understanding various graph theory concepts and measures. Selecting this set of regions from our thresholded matrix:

In [ ]:
A = thresh_cmat[:15, :15]
region_labels = [x[-5:] for x in labels[:15]]

fig, ax = plt.subplots(figsize=(5, 5))
plotting.plot_matrix(A, vmax=1, vmin=-1, labels=region_labels, figure=fig)

In graph theory speak, each region is a **node**. And each corretion between regions/nodes are called **edges**; the strength of the correlation is the edge **weight**. This connectivity structure can be represented in a graph. First we need to build our graph in networkx (`nx`). We can use our connectivity matrix, called an **adjacency matrix** in graph theory (`A`), to create `G`, our graph. We'll label our nodes according to the regions. 

In [ ]:
G = nx.from_numpy_matrix(A)
G = nx.relabel_nodes(G, lambda x: region_labels[x])

### 2.1 Plotting our graph

Now we can plot our graph. A really common way to plot graphs are using force-directed drawing algorithms that try to position nodes such that the edges are equal in length and minimal edge overlap (see [Wikipedia](https://en.wikipedia.org/wiki/Force-directed_graph_drawing)). Networkx has the Kamada-Kawai algorithm: 

In [ ]:
region_colors = ['purple'] * 9 + ['steelblue'] * 6

nx.draw_kamada_kawai(G, node_color=region_colors, node_size=1000, 
                     with_labels=True, font_color='w')

### 2.2 Adding weights

Above, we're completely ignoring weights/correlations. We can add these in, but we'll have to program this differently so we can customize it a bit.

First, we can extract out our edges and their corresponding weights. This is akin to going through each cell above and getting the corresponding regions, and the correlation value.

In [ ]:
edges, weights = zip(*nx.get_edge_attributes(G,'weight').items())
edges

In [ ]:
weights

In [ ]:
weight_widths = [(.5 + x) ** 4 for x in weights]


layout = nx.kamada_kawai_layout(G)
nx.draw_networkx_nodes(G, layout, node_color=region_colors, node_size=1000, 
                       with_labels=True)
nx.draw_networkx_labels(G, layout, font_color='w')
nx.draw_networkx_edges(G, layout, edgelist=edges, width=weight_widths)
plt.axis('off');

### 2.3 Network density

We can measure how dense each network is by dividing the number of edges by the number of possible edges. In other words, **network density** is the fraction of possible edges that exist.   

In [ ]:
vis_density, n, k = bct.density_und(A[:9, :9])
mot_density, n, k = bct.density_und(A[9:, 9:])

fig, ax = plt.subplots(figsize=(2, 3))
ax.bar(['Vis', 'Mot'], [vis_density, mot_density],
       width=1, color=['purple', 'steelblue'])
sns.despine()

By looking at our connectivity matrix, we can confirm these results. For instance, all of the motor regions are connected with one another, hence a density of 1. 

### 2.4 Node degree and strength

**Node degree** is simply the number of edges for a node. If we didn't threshold our connectivity matrix, all nodes would have the same degree (i.e. number of regions - 1). Because we applied a threshold, certain nodes have greater degree than others. We can use Brain Connectivity Toolbox to compute the degrees of our nodes:  

In [ ]:
degrees = bct.degrees_und(A)

fig, ax = plt.subplots(figsize=(10, 3))
ax.bar(region_labels, degrees, width=.5, color=region_colors)
sns.despine()

Node degree ignores the connectivity weights (i.e. correlation). **Node strength**, however, is the sum of weights for a node. 

In [ ]:
strengths = bct.strengths_und(A)

fig, ax = plt.subplots(figsize=(10, 3))
ax.bar(region_labels, strengths, width=.5, color=region_colors)
sns.despine()

### 2.5 Centrality



In [ ]:
eigin_cent = bct.eigenvector_centrality_und(A)

fig, ax = plt.subplots(figsize=(10, 3))
ax.bar(region_labels, eigin_cent, width=.5, color=region_colors)
sns.despine()

In [ ]:
between_cent = bct.betweenness_wei(A)

fig, ax = plt.subplots(figsize=(10, 3))
ax.bar(region_labels, between_cent, width=.5, color=region_colors)
sns.despine()

### 2.6 Clustering and Modularity

In [ ]:
# get upper triangle
distances = spatial.distance.squareform(1 - A, checks=False)

# apply hierarchical clustering 
linkages = cluster.hierarchy.linkage(distances, method='average')

# plot dendogram
dendo = cluster.hierarchy.dendrogram(linkages, labels=region_labels)
plt.xticks(rotation=45);

In [ ]:
modulatiry = bct.modularity_und(A)

In [ ]:
bct.community_louvain(A)

## 3. Whole-brain analysis

In [ ]:
label_data = pd.DataFrame({'number': np.arange(len(labels)) + 1, 'region': labels})
label_data

In [ ]:
def set_color(x):
    
    networks = ['Vis', 'SomMot', 'DorsAttn', 'SalVentAttn', 
                'Limbic', 'Cont', 'Default']
    cmap = ['purple', 'steelblue', 'green', 'violet', 
            'lightgoldenrodyellow', 'orange', 'indianred']
    pairings = dict(zip(networks, cmap))
    
    network_label = x.split('_')[2]
    return pairings[network_label]

node_colors = [set_color(i) for i in labels]

In [ ]:
G = nx.from_numpy_matrix(thresh_cmat)
G = nx.relabel_nodes(G, lambda x: label_data['number'].tolist()[x])



In [ ]:
nx.draw_kamada_kawai(G, node_color=node_colors, node_size=150)

In [ ]:
nx.draw_kamada_kawai(G, node_color=node_colors, node_size=150, with_labels=True)

For reference, the Schaefer atlas layout is below:

In [ ]:
nx.draw_networkx?

## 3. Graph theoretical measures

### 3.1 Node degree 

### 3.2. Node centrality

### 3.3. 